In [1]:
import tensorflow as tf
import numpy as np
import os
import time


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [2]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

import textract
text = textract.process("D:\\Nico\\OneDrive\\Studium\\Bachelor\\Bachelor Thesis\\Abschlussarbeit_Nico_Wellermann.pdf").decode("utf-8")

print(f'Length of text: {len(text)} characters')

Length of text: 135349 characters


In [6]:
print(text[11500:12000])

nderem versucht, dem Computer die menschliche Sprache zu
vermitteln. Bei virtuellen Marktplätzen hat sich seit der Einführung viel verändert. So
wird z.B. das Kundenverhalten mithilfe neuronaler Netze analysiert, um gezielt Vorschläge
zu platzieren. Die eigentliche Produktsuche hat sich hingegen kaum verändert. In dieser
Arbeit wird ein alternatives Verfahren untersucht, das mithilfe der natürlichen Sprachverarbeitung die herkömmliche Produktsuche ersetzen soll.

1.1. Motivation
In


In [7]:
vocab = sorted(set(text))
print(f'Einzigartige Chars: {len(vocab)}')

Einzigartige Chars: 102


In [8]:
# Zwei Lookup tables, chars -> zahlen & zahlen -> chars
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) 

In [9]:
for char, _ in zip(char2idx, range(20)):
    print(f'{repr(char)}: {char2idx[char]}')

'\n': 0
'\x0c': 1
'\r': 2
' ': 3
'!': 4
'%': 5
'&': 6
'(': 7
')': 8
'+': 9
',': 10
'-': 11
'.': 12
'/': 13
'0': 14
'1': 15
'2': 16
'3': 17
'4': 18
'5': 19


In [10]:
# Wie ist der Text mapped?
print(f'{repr(text[:13])} ------ mapped ------ > {text_as_int[:13]}')

'Name:\r\n\r\nNico' ------ mapped ------ > [43 59 71 63 24  2  0  2  0 43 67 61 73]


In [11]:
# Das Trainieren ist relativ einfach
# Wir versuchen ein vorhersage Modell auf Char basis. Wir haben bereits einen (langen) original Text. 
# Wir gehen einfach für jeden char durch das Modell und können testen ob es richtig vorhersagt, da wir den Text haben
# RNN haben den Vorteil, einen internen Zustand zu haben, der das vorherige Ergebniss speichert. Dadurch gut für Wörter / Sätze geeignet

# Wir trainieren das Modell in Sequenzen. Bsp. Seq_leng = 4 und der Text = "Hello" input = "Hell" output = "ello"

seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)
print(examples_per_epoch)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

1340
N
a
m
e
:


In [16]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [20]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

#for input_example, target_example in  dataset.take(1):
#    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
#    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

In [21]:
# Example copy pasta wie das Modell lernen würde:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 43 ('N')
  expected output: 59 ('a')
Step    1
  input: 59 ('a')
  expected output: 71 ('m')
Step    2
  input: 71 ('m')
  expected output: 63 ('e')
Step    3
  input: 63 ('e')
  expected output: 24 (':')
Step    4
  input: 24 (':')
  expected output: 2 ('\r')


In [22]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [23]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 102) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           26112     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 102)           104550    
Total params: 4,068,966
Trainable params: 4,068,966
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 71,  93,  31,  55,  75,  30,  44,  23,  38,  16,  39,  15,  92,
        26,  56,  14,   2,  70,  95,  27,  88,  62,  26,  82,  18,  17,
        40,  12,  27,  70,  90,  29,  74,  92,   0,  61,  25,  77,  62,
        69,  16,  16,  72,  63,  38,  70,  52,  75,  61,  26,  63,  28,
        27,  66,  92,  31,  16,  42,  67,  32,  86,  68,  92,  61,   2,
        78, 101,  53,  94,  65,  73,  79,  35,  88,   3,  94,  25,  75,
        71,  31,  78,  36,  86,  85,   0,  54,  74,  75,   7,  37,  10,
        58,  69,  63,  84,  23,  82,  94,   8,  65], dtype=int64)

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' . . .\r\n3.3.4 Persistierung . . . . . . . . . . .\r\n3.3.5 Auswertung . . . . . . . . . . . .\r\n3.4 Pro'

Next Char Predictions: 
 'm̈BZqAO9I2J1́<[0\rl–=äd<x43K.=lü?ṕ\nc;sdk22neIlWqc<e>=h́B2MiC}j́c\rt•X̌gouFä ̌;qmBtG}{\nYpq(H,]kez9x̌)g'


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print(example_batch_predictions.shape)
print(example_batch_loss.numpy().mean())

(64, 100, 102)
4.6237774


In [24]:
model.compile(optimizer="adam", loss=loss)

In [25]:
# Da das Training vermutlich sehr lange dauert, wollen wir Checkpoints haben.
checkpoint_dir = './nico_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
20/20 [==============================] - 2s 79ms/step - loss: 4.4151
Epoch 2/10
20/20 [==============================] - 1s 71ms/step - loss: 3.1985
Epoch 3/10
20/20 [==============================] - 2s 76ms/step - loss: 2.7310
Epoch 4/10
20/20 [==============================] - 1s 75ms/step - loss: 2.4918
Epoch 5/10
20/20 [==============================] - 1s 72ms/step - loss: 2.3616
Epoch 6/10
20/20 [==============================] - 1s 73ms/step - loss: 2.2606
Epoch 7/10
20/20 [==============================] - 1s 74ms/step - loss: 2.1793
Epoch 8/10
20/20 [==============================] - 1s 72ms/step - loss: 2.1002
Epoch 9/10
20/20 [==============================] - 1s 72ms/step - loss: 2.0162
Epoch 10/10
20/20 [==============================] - 1s 69ms/step - loss: 1.9386


In [26]:
# Um die Batch Size zu ändern muss dass Modell neu erzeugt werden, mit den Weights aus dem Training
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            26112     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 102)            104550    
Total params: 4,068,966
Trainable params: 4,068,966
Non-trainable params: 0
_________________________________________________________________


In [27]:
def generate_text(model, start_string, temperature = 1.0):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = temperature # Tief = mehr vorhersehbar, Hoch = mehr unerwartet
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"Einleitung "))

Einleitung giesbeieren västame mutzissen zzionnachen
bieken mmabeienen em ehsweit, nachen
Sechnioßen. HKallsist die hiasches Anwendute teis die Lehoresist die s.

Abberdeben. Abwirdungale vorhel en Aus alt riendanf fathle enst wird–Klichten ein Terestelonon die erseitisto — wurden.

2.337

Raweis
Bebinduzr, wurd einse metzein kelfener Asföllss, wird. Die istrveiteignwichtetichen.
Eingerach “2001). Rachlitionasen Preiode fügle faturient
weiss e kohnessarn. Domicheinss ins undung däs verwenl Ans dasse ze werdensscheit undet.
Dieserbeite zufesteplen der Derännache htries eine enfandenan stwitpe Abbeitzuge orbebenfen on de
Be riegen sschise reiken die enn rimm tor von Ethnach uftor Autigrigk–ersommäbvioren in dee Anworden Das
betalden des An der Algchlebsichter äcwerkeretthr der Sutzresxenen.
mir aphacbeu hriscenitreitend, verharinit zuher gehauf des Vestetztan. Dasis eanen Ander unde anfüglon wie der Ssteill nellet
diesen werch dan Lodimt: Ang–den Ergexchmitt eingager :. Mere


In [29]:
# Advanced Training:
model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(target, predictions, from_logits=True))
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [30]:
# Training step
EPOCHS = 60

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.624759197235107
Epoch 1 Loss 3.6770
Time taken for 1 epoch 2.097106695175171 sec

Epoch 2 Batch 0 Loss 3.6241683959960938
Epoch 2 Loss 2.8968
Time taken for 1 epoch 0.9770314693450928 sec

Epoch 3 Batch 0 Loss 2.870112895965576
Epoch 3 Loss 2.5338
Time taken for 1 epoch 0.9730319976806641 sec

Epoch 4 Batch 0 Loss 2.511029005050659
Epoch 4 Loss 2.4738
Time taken for 1 epoch 1.0231082439422607 sec

Epoch 5 Batch 0 Loss 2.3689935207366943
Epoch 5 Loss 2.3287
Time taken for 1 epoch 1.259098768234253 sec

Epoch 6 Batch 0 Loss 2.2994742393493652
Epoch 6 Loss 2.2235
Time taken for 1 epoch 0.9943468570709229 sec

Epoch 7 Batch 0 Loss 2.2322096824645996
Epoch 7 Loss 2.1111
Time taken for 1 epoch 0.9850912094116211 sec

Epoch 8 Batch 0 Loss 2.0730948448181152
Epoch 8 Loss 2.1221
Time taken for 1 epoch 0.9805796146392822 sec

Epoch 9 Batch 0 Loss 2.1575448513031006
Epoch 9 Loss 1.9125
Time taken for 1 epoch 0.9806873798370361 sec

Epoch 10 Batch 0 Loss 1.8859862089157104
E

In [34]:
import tensorflow_docs as tfdocs
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(history)
#plt.ylim([0.5, 0.7])

ModuleNotFoundError: No module named 'tensorflow_docs'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()
print(generate_text(model, start_string=u"Einleitung: ", temperature = 1.0))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            26112     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 102)            104550    
Total params: 4,068,966
Trainable params: 4,068,966
Non-trainable params: 0
_________________________________________________________________
Einleitung: Lolgat Klasse; F. 25.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.

45
38
40
31
53

.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.
.
.

.
.
.
.
.
.
.

In [44]:
print(generate_text(model, start_string=u"Umsetzung ", temperature = 0.5))

Umsetzung der Eingabe verloren,
die bei einer Auswertung nicht beachtet werden sollen (hier Fenstergröße = 1). Die Anforderungen werden mit den Buchstaben FA und NA für fin
Outpuell ausgewichtet und das e genaueren Evaluation der Daten (Abbildung 14) zeigte, dass das Modell keine Produkte erkennt, die ausgewählte Sequenz dem jeweiligen Attribut
zugewiesen werden sollten, was die Bearbeitung von Smartphones miteinschließt. Das häufige Eingeben von Text wird durch die Verwendung einer Anfrage und so die Performance zu verbessern. Die Verfahren werden
passend zu den Klassen “Erkannt, rient ein Ungleichgewicht und Sequenzen
mit stark variierender Länge werden von dem Modell nur schwerer Vorverarbeitung die herkömmliche Produktsuche erschwert. Abbildung 8 zeigt eine beispielhafte Eingabe in der die Attribute farblich hervorgehoben werden. In dem 1.1. Kapitel wurde die Zahlen betrachtet, welches die Anwender selbst die Attribute markieren, um die Datenbasis zu erweitern. Technologische

In [1]:
print("hello")

hello


b'Name:\r\n\r\nNico Wellermann\r\n\r\nThema:\r\n\r\nNat\xc3\xbcrlichsprachliche Kommunikation in virtuellen\r\nMarktpl\xc3\xa4tzen\r\n\r\nArbeitsplatz:\r\n\r\nCAS Software AG, Karlsruhe\r\n\r\nReferent:\r\n\r\nProf. Dr. W\xc3\xb6lfel\r\n\r\nKorreferent:\r\n\r\nProf. Dr. K\xc3\xb6rner\r\n\r\nAbgabetermin:\r\n\r\n09.02.2020\r\n\r\nKarlsruhe, 09.11.2019\r\nDer Vorsitzende des Pr\xc3\xbcfungsausschusses\r\n\r\nProf. Dr. Heiko K\xc3\xb6rner\r\n\r\nFakult\xc3\xa4t f\xc3\xbcr Informatik und Wirtschaftsinformatik\r\n\r\nBachelor-Thesis\r\n\r\n\x0cErkla\xcc\x88rung\r\nHiermit versichere ich, dass ich meine Abschlussarbeit selbsta\xcc\x88ndig verfasst und keine\r\nanderen als die angegebenen Quellen und Hilfsmittel benutzt habe.\r\n\r\nDatum:\r\n\r\n.......................................................\r\n(Unterschrift)\r\n\r\n\x0cZUSAMMENFASSUNG\r\n\r\nI. Zusammenfassung\r\nVirtuelle Marktpla\xcc\x88tze werden von Jahr zu Jahr bedeutsamer. Auf elektronischen Marktpla\xcc\x88tzen beispielsweis

In [4]:
print(text)

Name:

Nico Wellermann

Thema:

Natürlichsprachliche Kommunikation in virtuellen
Marktplätzen

Arbeitsplatz:

CAS Software AG, Karlsruhe

Referent:

Prof. Dr. Wölfel

Korreferent:

Prof. Dr. Körner

Abgabetermin:

09.02.2020

Karlsruhe, 09.11.2019
Der Vorsitzende des Prüfungsausschusses

Prof. Dr. Heiko Körner

Fakultät für Informatik und Wirtschaftsinformatik

Bachelor-Thesis

Erklärung
Hiermit versichere ich, dass ich meine Abschlussarbeit selbständig verfasst und keine
anderen als die angegebenen Quellen und Hilfsmittel benutzt habe.

Datum:

.......................................................
(Unterschrift)

ZUSAMMENFASSUNG

I. Zusammenfassung
Virtuelle Marktplätze werden von Jahr zu Jahr bedeutsamer. Auf elektronischen Marktplätzen beispielsweise finden viele Käufer–Verkäufer Situationen statt. Anwender erstellen
ein digitales Angebot, das von potenziellen Käufern gefunden werden möchte. Für eine intuitive Bedienung soll die Verwendung der natürlichen Eingabe unter